### A program for selecting PS pixels in multi-temporal InSAR

### Input: A stack of interferograms in .mat format (exported from WabInSAR software developed by Manoochehr Shirzaei). WabInSAR software is open access and can be downloaded from the following link:
### https://sites.google.com/vt.edu/eadar-lab/software



### The interferograms are first divided into image patches of 100 by 100 pixels and then fed to the network

### Output: A map with labels 0 and 1, 0 denoting non-PS pixels, and 1 denoting PS pixels 



### Loading essential libraries

In [ ]:
import numpy as np
import scipy.io as spio
import pandas
import hdf5storage as hs
import mat73

In [ ]:
import tensorflow
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, Conv2DTranspose, MaxPooling2D, UpSampling2D

In [ ]:
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import argparse
import keras.backend as K

In [ ]:
import numpy as np
from patchify import patchify, unpatchify
from PIL import Image

### Download training data from the following link:

https://drive.google.com/file/d/1UXHPNw8F9kMPXvPg42cO6dveyAahWMAk/view?usp=sharing

### Unzip data and store locally in your computer

## Define path to training data

In [ ]:
fpath='/media/ashutosh/Data/Deep_learning_InSAR/Datasets/Example_dataset' #enter your local data path here

## Load training data

In [ ]:
dataset=hs.loadmat(fpath+'/ph_im.mat')

In [ ]:
X=dataset['ph_im']

### The program works well for time series of 20-25 interferograms

In [ ]:
X=X[:,:, ::5]  #keep a sampling interval to keep about 20-25 time steps

### Load training labels

In [ ]:
labels=spio.loadmat(fpath+'/elpx.mat')

In [ ]:
y=labels['elpx_imloc']

In [ ]:
Xrow=X.shape[0]-X.shape[0]%100
Xcol=X.shape[1]-X.shape[1]%100

In [ ]:
X1=X[0:Xrow, 0:Xcol]

In [ ]:
y1=y[0:Xrow, 0:Xcol]

In [ ]:
img_rows, img_cols=100, 100

In [ ]:
s=100 #step_size for image division

### Divide whole image into patches 

In [ ]:
hk=patchify(X1, (img_rows,img_cols,X1.shape[2]), step=s) ## for training inputs

In [ ]:
gn=patchify(y1, (img_rows,img_cols), step=s) ## for training labels

In [ ]:
patches=hk

In [ ]:
print(patches.shape)

In [ ]:
patchesy=gn

In [ ]:
patch=np.zeros((patches.shape[0]*patches.shape[1],patches.shape[3],patches.shape[4],patches.shape[5]))

In [ ]:
for i in range(0, patches.shape[0]):
    for j in range(0, patches.shape[1]):
        patch1 = patches[i, j, 0]
        patcht=patch1[:,:,0]
        plt.figure(i+1)
        num = i * patches.shape[1] + j
        print('num', num)
        patch[num, :,:,:]=patch1
        plt.imshow(patcht)
#         plt.close()

        
 
ele1=np.count_nonzero(patch)

In [ ]:
ele2=np.count_nonzero(X1)

In [ ]:
print('Number of non-zero pixels in divided image stack and full image are', ele1, ele2)

In [ ]:
patchy = np.zeros((patchesy.shape[0]*patchesy.shape[1],patchesy.shape[2],patchesy.shape[3]))

In [ ]:
for i in range(patchesy.shape[0]):
    for j in range(patchesy.shape[1]):
        patchy1 = patchesy[i, j, :,:]
        patchyt=patchy1
        plt.figure(i+1)
        numy = i * patches.shape[1] + j
        print('numy', numy)
        patchy[numy, :,:]=patchy1
        plt.imshow(patchyt)
#         plt.close()

        

In [ ]:
ele1=np.count_nonzero(patchy)

In [ ]:
ele2=np.count_nonzero(y1)

In [ ]:
print('Number of non-zero pixels in divided image stack and full image are', ele1, ele2)

In [ ]:
input_shape = (img_rows, img_cols, X1.shape[2])

In [ ]:
Xdiv=patch

In [ ]:
ydiv=patchy

## Dividing training data into train and test data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xdiv, ydiv, test_size = 0.2, random_state=32)

In [ ]:
X_train=np.moveaxis(X_train,3,1)

In [ ]:
X_test=np.moveaxis(X_test,3,1)

### Resizing training and test datasets to size (#samples, #timesteps, img_rows, img_cols,#bands) for convlstm input

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], img_rows, img_cols,1)

In [ ]:
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], img_rows, img_cols,1)

In [ ]:
y_train = y_train.reshape(y_train.shape[0], img_rows, img_cols, 1)

In [ ]:
y_test = y_test.reshape(y_test.shape[0], img_rows, img_cols, 1)

In [ ]:
Xdiv = np.moveaxis(Xdiv,3,1)

In [ ]:
Xdiv = Xdiv.reshape(Xdiv.shape[0], Xdiv.shape[1], img_rows, img_cols,1)

In [ ]:
ydiv = ydiv.reshape(ydiv.shape[0], img_rows, img_cols, 1)

In [ ]:
del dataset # dataset2, X,y

In [ ]:
import tensorflow as tf
print("tensorflow keras version:", tf.keras.__version__)
from keras.utils import np_utils
import keras.backend as K
from itertools import product

## Defining loss functions for the model

In [ ]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
def f1_loss(y_true, y_pred):
    smooth=100
    intersection = tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    f1 = (2 * intersection + smooth) / ( denominator + smooth)
   
    return (1 - f1) * smooth

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, Input, Reshape, Flatten, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers import LSTM
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D
from keras.models import Sequential, load_model
from keras_layer_normalization import LayerNormalization
from keras.layers import AveragePooling3D, Reshape, Activation, Flatten, Dense
from keras import backend as K

In [ ]:
batch_size=32
convlstm_model=Sequential()

In [ ]:
convlstm_model.add(ConvLSTM2D(filters=16, kernel_size=(2,5), input_shape=(None,img_rows,img_cols,1), padding='same', return_sequences=True, activation='relu'))
convlstm_model.add(LayerNormalization())

In [ ]:
convlstm_model.add(ConvLSTM2D(filters=16, kernel_size=(5,5), padding='same', return_sequences=False, activation='relu'))
convlstm_model.add(BatchNormalization())
# clstm_iss.add(LayerNormalization())

In [ ]:
convlstm_model.add(Conv2D(filters=16, kernel_size=(11,11), padding='same', activation='relu'))

In [ ]:
convlstm_model.add(Dropout(0.4))

In [ ]:
convlstm_model.add(Dense(1, activation='sigmoid'))

In [ ]:
convlstm_model.summary()

In [ ]:
from keras.optimizers import SGD, Adam

In [ ]:
opt1 = SGD(learning_rate=0.01, decay=1e-4, momentum=0.9, nesterov=True)
opt2 = Adam(learning_rate=0.001, decay=1e-4)
opt3 = Adam(learning_rate=0.01)

In [ ]:
convlstm_model.compile(optimizer='adam', loss=f1_loss, metrics=[f1])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=5)

In [ ]:
convlstm_model.fit(X_train, y_train, epochs=20, validation_data=[X_test, y_test], callbacks=early_stopping_monitor)

In [ ]:
ypred_test=convlstm_model.predict(X_test)

In [ ]:
ypred_full=convlstm_model.predict(Xdiv)

In [ ]:
ypred_test_class=np.round(ypred_test)   #convert probailities to labels

In [ ]:
ypred_full_class=np.round(ypred_full)   #convert probailities to labels for full dataset

In [ ]:
Xdiv=patches
image_height=X1.shape[0]
image_width=X1.shape[1]
patch_height=100
patch_width=100
channel_count=X.shape[2]

In [ ]:
output_patches=Xdiv
output_shape = (X1.shape[0], X1.shape[1], channel_count)
X_combined = unpatchify(output_patches, output_shape)

In [ ]:
ypred_full=ypred_full_class

In [ ]:
out_shapey=(y1.shape[0], y1.shape[1])
y_full=ypred_full.reshape(patchesy.shape[0], patchesy.shape[1], img_rows, img_cols)
y_combined=unpatchify(y_full, out_shapey)

In [ ]:
y_combined=y_combined.astype(np.uint8)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(y_combined, interpolation='nearest')
plt.show()

In [ ]:
plt.imshow(y1, interpolation='nearest')
plt.show()

In [ ]:
import scipy.io as spio

In [ ]:
spio.savemat(fpath+'/EastCoast_48_79_84/rs_convlstm_100by100_wabinsar.mat', dict(y1=y1, ypred_test_class=ypred_test_class,
                                                                             ypred_full=ypred_full, ydiv=ydiv, 
                                                                             y_test=y_test, y_train=y_train, 
                                                                             y_combined=y_combined))

### Saving trained model

In [ ]:
from keras.models import model_from_json

In [ ]:
classifier_json = convlstm_model.to_json()
with open("rs_convlstm_100by100_wabinsar.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5


# Saving trained model weights

convlstm_model.save_weights("rs_convlstm_100by100_wabinsar.tf",save_format='tf')

### Open trained model and associated weights

In [ ]:
json_file = open('rs_convlstm_100by100_wabinsar.json', 'r')
loaded_classifier_json = json_file.read()
json_file.close()
loaded_classifier = model_from_json(loaded_classifier_json)
# load weights into new classifier
loaded_classifier.load_weights("rs_convlstm_100by100_wabinsar.tf")
print("Loaded classifier from disk")

### Use loaded_classifier.predict(X_pred) to use existing model to predict labels from new dataset X_pred. The dimension of X_pred should also be (#samples, #timesteps, img_rows, img_cols,#bands), where img_rows, img_cols and #bands need to be same as that in the training data.

######################################################################################

## The program gives a map of PS and non-PS pixels. It also gives a model with saved weights that can be further trained on new datasets


Acknowledgements

Conceptualization, input data preparation, validation and funding: Manoochehr Shirzaei

Conceptualization for earlier versions: Avadh Bihari Narayan

Funding support for earlier version: Onkar Dikshit
    